In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
classes = ["Balcony", "Bar", "Bathroom", "Bedroom", "Business Centre", "Dining room", "Exterior",
           "Gym", "Living room", "Lobby", "Patio", "Pool", "Restaurant", "Sauna", "Spa"]


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
num_epochs = 20
batch_size = 32
learning_rate = 0.001


In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet statistics
])

In [15]:
dataset = ImageFolder("../input/hotel-images/Dataset/", transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

print("Train Size : " + str(train_size));
print("Test Size : " + str(test_size))

Train Size : 59196
Test Size : 14799


In [16]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


In [23]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.features = torchvision.models.alexnet(pretrained=True)
        num_features = self.features.classifier[6].in_features
        self.features.classifier[6] = nn.Linear(num_features, len(classes))
    
    def forward(self, x):
        x = self.features(x)
        return x

model = CNNModel().to(device)

S:\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
S:\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [25]:
total_step = len(train_loader)
print("Total Steps : " + str(total_step))

Total Steps : 1850


In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_correct = 0
    total_samples = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

        # Print training progress
        if (i + 1) == total_step:
            print(f"Epoch [{epoch + 1}/{num_epochs}]"
                  f"Train Loss: {loss.item():.4f}, Train Accuracy: {(total_correct / total_samples) * 100:.2f}%")
            
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0

                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
        
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
        
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
    
                print(f"Test Accuracy: {(100 * correct / total):.2f}%")
                print("")
        
        if epoch == num_epochs - 1:
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                all_predicted = []
                all_labels = []

                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
    
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
    
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    # Collect predicted and true labels for classification report
                    all_predicted.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

            avg_precision = precision_score(all_labels, all_predicted, average='macro')
            avg_recall = recall_score(all_labels, all_predicted, average='macro')
            avg_f1_score = f1_score(all_labels, all_predicted, average='macro')

            print(f"Average Precision: {avg_precision:.4f}")
            print(f"Average Recall: {avg_recall:.4f}")
            print(f"Average F1-Score: {avg_f1_score:.4f}")
            print("")

Epoch [1/20] Train Loss: 0.7095, Train Accuracy: 58.36%
Test Accuracy: 49.42%

Epoch [2/20] Train Loss: 0.4733, Train Accuracy: 65.27%
Test Accuracy: 58.76%

Epoch [3/20] Train Loss: 0.3995, Train Accuracy: 70.91%
Test Accuracy: 62.11%

Epoch [4/20] Train Loss: 0.2889, Train Accuracy: 75.41%
Test Accuracy: 66.82%

Epoch [5/20] Train Loss: 0.2643, Train Accuracy: 79.31%
Test Accuracy: 71.42%

Epoch [6/20] Train Loss: 0.2275, Train Accuracy: 83.43%
Test Accuracy: 75.67%

Epoch [7/20] Train Loss: 0.1873, Train Accuracy: 86.64%
Test Accuracy: 80.62%

Epoch [8/20] Train Loss: 0.1585, Train Accuracy: 88.32%
Test Accuracy: 85.42%

Epoch [9/20] Train Loss: 0.1435, Train Accuracy: 89.34%
Test Accuracy: 87.12%

Epoch [10/20] Train Loss: 0.1448, Train Accuracy: 89.78%
Test Accuracy: 88.36%

Epoch [11/20] Train Loss: 0.1398, Train Accuracy: 90.56%
Test Accuracy: 87.89%

Epoch [12/20] Train Loss: 0.1417, Train Accuracy: 90.13%
Test Accuracy: 88.12%

Epoch [13/20] Train Loss: 0.1365, Train Accuracy:

In [30]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f"Test Accuracy: {(100 * correct / total):.2f}%")

Validation Accuracy: 89.71%


In [28]:
model_path = "alexnet.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved at: {model_path}")

Model saved at: alexnet.pth
